In [ ]:
import pandas as pd

%pip install openpyxl

df = pd.read_excel('./douban.xlsx')
df.info()
# df.head(n=10)

In [ ]:
selection = df[['标题','链接','创建时间','我的评分','评论']].copy()
selection.rename(
    columns={
    '我的评分':'Rating', 
    '创建时间':'WatchedDate', 
    '标题':'Title', 
    '链接':'DoubanUrl', 
    '评论': 'Review'
    }, inplace=True)
selection.WatchedDate = selection.WatchedDate.apply(lambda x: x[:10])
selection.head(n=10)

In [ ]:
# query by movie id, to see which movie is missing

movieid = 'https://movie.douban.com/subject/1309046/'

selection[selection.apply(
    lambda row: row.astype(str).str.contains(movieid).any(), axis=1
    )]

In [ ]:
movie = pd.read_csv('./movie.csv', header=None)
movie.rename(columns={0:'Title', 1:'Rating', 2:'imdbID'}, inplace=True)
movie.head(n=10)
print(f'total number of movies: {len(movie)}')

movie.dropna(inplace=True)
print(f'total number of movies after dropping NaN: {len(movie)}')
movie.head(n=10)

In [ ]:
# merge and save

merged_df = pd.merge(movie, selection, left_on='Title', right_on='DoubanUrl', how='left')
final_df = merged_df[['imdbID', 'WatchedDate', 'Rating_x', 'Review']]
final_df.rename(columns={'Rating_x':'Rating'}, inplace=True)
final_df.to_csv('./final.csv', index=False)

In [ ]:
# add tail message to reviews

tail_message = "\n\n@imported from douban.com <>"

final_df_with_signs = final_df.copy()

for index, row in final_df_with_signs.iterrows(): #iterrows() returns a tuple (index, Series) for each row
    if pd.notna(row.Review): # check if entry is NaN
        final_df_with_signs.at[index, 'Review'] += "\n\n@imported from douban.com <" + row.WatchedDate + ">"
        # here, row.Review += '' will not work, because `row` is a copy of the series generated by iterrows(),
        # and it is not the original series

final_df_with_signs.to_csv('./final_with_signs.csv', index=False)